In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [2]:
sentence_df = pd.read_csv("/data/semeval-2020/task-11/processed/sentence_dataset_v2.csv")

In [3]:
sentence_df.head(5)

,article_id,sentence_id,sentence,has_propaganda,cleaned_sentence
0,762956953,0,Iran Admits To Aiding Al-Qaeda and Facilitatin...,False,Iran Admits To Aiding Al-Qaeda and Facilitatin...
1,762956953,1,But now we have definitive confirmation.,False,But now we have definitive confirmation.
2,762956953,2,It was Iran Bush should have invaded after 9/1...,False,It was Iran Bush should have invaded after 9/1...
3,762956953,3,"Now consider this: even though, as President o...",False,"Now consider this: even though, as President o..."
4,762956953,4,The Iran nuclear deal should never have procee...,True,The Iran nuclear deal should never have procee...


In [4]:
sentence_df["propaganda"] = sentence_df["has_propaganda"].apply(lambda x:1 if x else 0)

In [5]:
sentence_df.head(5)

,article_id,sentence_id,sentence,has_propaganda,cleaned_sentence,propaganda
0,762956953,0,Iran Admits To Aiding Al-Qaeda and Facilitatin...,False,Iran Admits To Aiding Al-Qaeda and Facilitatin...,0
1,762956953,1,But now we have definitive confirmation.,False,But now we have definitive confirmation.,0
2,762956953,2,It was Iran Bush should have invaded after 9/1...,False,It was Iran Bush should have invaded after 9/1...,0
3,762956953,3,"Now consider this: even though, as President o...",False,"Now consider this: even though, as President o...",0
4,762956953,4,The Iran nuclear deal should never have procee...,True,The Iran nuclear deal should never have procee...,1


In [6]:
sentence_df[sentence_df["cleaned_sentence"].isna() == True]

,article_id,sentence_id,sentence,has_propaganda,cleaned_sentence,propaganda
1068,735855251,19,6-/span>,True,NaN,1
1656,697959084,19,5-/span>,True,NaN,1
1802,999000147,24,5-/span>,True,NaN,1
7750,721890296,43,8-/span>,True,NaN,1
10547,705035735,21,8-/span>,True,NaN,1


In [7]:
len(sentence_df)

15565

In [8]:
sentence_df.dropna(inplace=True)

In [9]:
len(sentence_df)

15560

In [10]:
from nltk.corpus import stopwords 


from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words=stopwords.words("english"),ngram_range=(2,2))
vectorizer.fit(sentence_df["cleaned_sentence"].values.tolist())




TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(2, 2), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [11]:
train_tfidf_features = vectorizer.transform(sentence_df["cleaned_sentence"].values.tolist())

In [12]:
len(vectorizer.get_feature_names())

131434

In [13]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

In [14]:
skf = StratifiedKFold(n_splits=4)
y = sentence_df["propaganda"].values

In [15]:
for train_index,test_index in skf.split(train_tfidf_features,y):
    clf = LinearSVC(random_state=0,max_iter=10000,verbose=2)
    clf.fit(train_tfidf_features[train_index].toarray(),y[train_index])
    print(classification_report(clf.predict(train_tfidf_features[test_index].toarray()),y[test_index]))


[LibLinear]              precision    recall  f1-score   support

           0       0.98      0.70      0.81      3731
           1       0.09      0.68      0.15       159

    accuracy                           0.70      3890
   macro avg       0.53      0.69      0.48      3890
weighted avg       0.94      0.70      0.79      3890

[LibLinear]              precision    recall  f1-score   support

           0       0.98      0.69      0.81      3761
           1       0.07      0.64      0.12       129

    accuracy                           0.69      3890
   macro avg       0.52      0.66      0.47      3890
weighted avg       0.95      0.69      0.79      3890

[LibLinear]              precision    recall  f1-score   support

           0       0.97      0.69      0.81      3686
           1       0.09      0.55      0.16       204

    accuracy                           0.69      3890
   macro avg       0.53      0.62      0.48      3890
weighted avg       0.92      0.69      0.